In [1]:
# !apt-get install -y \
#     libgl1-mesa-dev \
#     libgl1-mesa-glx \
#     libglew-dev \
#     libosmesa6-dev \
#     software-properties-common

# !apt-get install -y patchelf

# !apt-get update --fix-missing
# !pip install stable-baselines3
# !pip install mujoco
# !pip install  --upgrade gymnasium==0.29
# !pip install free-mujoco-py

In [2]:
import os
import time
import wandb
import random

import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from stable_baselines3.common.buffers import ReplayBuffer

In [3]:
def make_env(env_id, seed, idx, capture_video, run_name):
    def thunk():
        if capture_video and idx == 0 :
            env = gym.make(env_id, render_mode="rgb_array")
            env = gym.wrappers.RecordVideo(env, f"videos/{run_name}")
        else:
            env = gym.make(env_id)

        env = gym.wrappers.RecordEpisodeStatistics(env)
        env.action_space.seed(seed)
        env.observation_space.seed(seed)
        return env
    return thunk

In [4]:
class QNetwork(nn.Module):

    def __init__(self, env):
        super().__init__()
        self.fc1 = nn.Linear(np.array(env.single_observation_space.shape).prod() + np.prod(env.single_action_space.shape), 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 1)

    def forward(self, x, a):
        x = torch.cat([x, a], 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [5]:
class Actor(nn.Module):

    def __init__(self, env):
        super().__init__()
        self.fc1 = nn.Linear(np.array(env.single_observation_space.shape).prod(), 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc_mu = nn.Linear(256, np.prod(env.single_action_space.shape))
        self.register_buffer(
            "action_scale", torch.tensor((env.action_space.high - env.action_space.low) / 2.0, dtype=torch.float32)
        )
        self.register_buffer(
            "action_bias", torch.tensor((env.action_space.high + env.action_space.low) / 2.0, dtype=torch.float32)
        )

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = torch.tanh(self.fc_mu(x))

        return x * self.action_scale + self.action_bias

In [6]:
def train(
    env_id,
    seed,
    total_timesteps,
    learning_rate,
    buffer_size,
    gamma,
    tau,
    batch_size,
    exploration_noise,
    learning_starts,
    policy_frequency,
    noise_clip):

    run_name = f"{env_id}__{seed}__{int(time.time())}"
    wandb.init(
        project="ddpg-mujoco-benchmark",
        config={
            "env":env_id,
            "seed":seed,
            "timesteps":total_timesteps,
            "lr":learning_rate,
            "buffer_size":buffer_size,
            "gamma":gamma,
            "tau": tau,
            "batch_size": batch_size,
            "exploration_noise":exploration_noise,
            "learning_starts":learning_starts,
            "policy_frequency":policy_frequency,
            "noise_clip":noise_clip,
        },
        sync_tensorboard=True,
        monitor_gym=True,
        name=run_name
    )
    writer = SummaryWriter(f"runs/{run_name}")

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    envs = gym.vector.SyncVectorEnv(
        [make_env(env_id, seed, 0, True, run_name)]
    )

    actor = Actor(envs).to(device)
    qf1 = QNetwork(envs).to(device)
    qf1_target = QNetwork(envs).to(device)
    target_actor = Actor(envs).to(device)
    target_actor.load_state_dict(actor.state_dict())
    qf1_target.load_state_dict(qf1.state_dict())

    q_optimizer = optim.Adam(qf1.parameters(), lr=learning_rate)
    actor_optimizer = optim.Adam(actor.parameters(), lr=learning_rate)

    envs.single_observation_space.dtype = np.float32
    rb = ReplayBuffer(
        buffer_size,
        envs.single_observation_space,
        envs.single_action_space,
        device,
        handle_timeout_termination=False,
    )

    start_time = time.time()
    obs, _ = envs.reset(seed=seed)

    for global_step in range(total_timesteps):
        if global_step < learning_starts:
            actions = np.array([envs.single_action_space.sample() for _ in range(envs.num_envs)])
        else:
            with torch.no_grad():
                actions = actor(torch.Tensor(obs).to(device))
                actions += torch.normal(0, actor.action_scale * exploration_noise)
                actions = actions.cpu().numpy().clip(envs.single_action_space.low, envs.single_action_space.high)

        next_obs, rewards, terminated, truncated, infos = envs.step(actions)
        if "final_info" in infos:
            for info in infos["final_info"]:
                print(f"global_step={global_step} episodic_return={info['episode']['r']}")
                writer.add_scalar("charts/episodic_return", info["episode"]["r"], global_step)
                writer.add_scalar("charts/episodic_length", info["episode"]["l"], global_step)
                break


        real_next_obs = next_obs.copy()
        for idx,d in enumerate(truncated):
            if d:
                real_next_obs[idx] = infos["final_observation"][idx]
        rb.add(obs, real_next_obs, actions, rewards, terminated, infos)

        obs = next_obs

        if global_step > learning_starts:
            data = rb.sample(batch_size)
            with torch.no_grad():
                next_state_actions = target_actor(data.next_observations)
                qf1_next_target = qf1_target(data.next_observations, next_state_actions)
                next_q_value = data.rewards.flatten() + (1 - data.dones.flatten()) * gamma * (qf1_next_target).view(-1)

            qf1_a_values = qf1(data.observations, data.actions).view(-1)
            qf1_loss = F.mse_loss(qf1_a_values, next_q_value)

            q_optimizer.zero_grad()
            qf1_loss.backward()
            q_optimizer.step()

            if global_step % policy_frequency == 0:
                actor_loss = -qf1(data.observations, actor(data.observations)).mean()
                actor_optimizer.zero_grad()
                actor_loss.backward()
                actor_optimizer.step()

                for param, target_param in zip(actor.parameters(), target_actor.parameters()):
                    target_param.data.copy_(tau * param.data + (1 -tau) * target_param.data)
                for param, target_param in zip(qf1.parameters(), qf1_target.parameters()):
                    target_param.data.copy_(tau * param.data + (1 -tau) * target_param.data)

            if global_step % 100 == 0:
                writer.add_scalar("losses/qf1_loss", qf1_loss.item(), global_step)
                writer.add_scalar("losses/actor_loss", actor_loss.item(), global_step)
                writer.add_scalar("losses/qf1_values", qf1_a_values.mean().item(), global_step)
                writer.add_scalar("charts/SPS", int(global_step/ (time.time()-start_time)), global_step)

    envs.close()
    writer.close()
    wandb.finish()


In [7]:
env = {"hopper":"Hopper-v2","humanoid":"Humanoid-v2","halfCheetah":"HalfCheetah-v2","ant":"Ant-v2"}
seed = 1
total_timesteps = 500000
learning_rate = 0.00003
buffer_size = 100000
gamma = 0.99
tau = 0.005
batch_size = 1024
exploration_noise = 0.1
learning_starts = 25000
policy_frequency = 2
noise_clip = 0.5

In [8]:
train(env["hopper"],seed, total_timesteps,
      learning_rate, buffer_size, gamma, tau, batch_size,
      exploration_noise,learning_starts,
      policy_frequency,noise_clip)

wandb: Currently logged in as: chkda. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.9/dist-packages/gymnasium/envs/registration.py:523: DeprecationWarning: WARN: The environment Hopper-v2 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(
/usr/local/lib/python3.9/dist-packages/gymnasium/envs/mujoco/mujoco_env.py:185: DeprecationWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.deprecation(
/usr/local/lib/python3.9/dist-packages/gymnasium/utils/passive_env_checker.py:164: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be <class 'numpy.float32'>, actual type: float64
  logger.warn(
/usr/local/lib/python3.9/dist-packages/gymnasium/utils/passive_env_checker.py:188: UserWarning: WARN: The obs returned b

Moviepy - Building video /notebooks/rl-algos/videos/Hopper-v2__1__1700234374/rl-video-episode-0.mp4.
Moviepy - Writing video /notebooks/rl-algos/videos/Hopper-v2__1__1700234374/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/videos/Hopper-v2__1__1700234374/rl-video-episode-0.mp4
global_step=12 episodic_return=[10.095653]
Moviepy - Building video /notebooks/rl-algos/videos/Hopper-v2__1__1700234374/rl-video-episode-1.mp4.
Moviepy - Writing video /notebooks/rl-algos/videos/Hopper-v2__1__1700234374/rl-video-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/videos/Hopper-v2__1__1700234374/rl-video-episode-1.mp4
global_step=33 episodic_return=[24.481318]
global_step=77 episodic_return=[30.460733]
global_step=97 episodic_return=[3.5976954]
global_step=113 episodic_return=[10.385685]
global_step=145 episodic_return=[26.22122]
global_step=167 episodic_return=[9.670132]
global_step=208 episodic_return=[31.524872]
Moviepy - Building video /notebooks/rl-algos/videos/Hopper-v2__1__1700234374/rl-video-episode-8.mp4.
Moviepy - Writing video /notebooks/rl-algos/videos/Hopper-v2__1__1700234374/rl-video-episode-8.mp4



...

Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/videos/Hopper-v2__1__1700234374/rl-video-episode-729.mp4
global_step=16251 episodic_return=[7.0388546]
global_step=16272 episodic_return=[15.929157]
global_step=16292 episodic_return=[14.264844]
global_step=16373 episodic_return=[121.38163]
global_step=16390 episodic_return=[9.459834]
global_step=16426 episodic_return=[18.502449]
global_step=16436 episodic_return=[7.615998]
global_step=16448 episodic_return=[9.339965]
global_step=16465 episodic_return=[11.2556925]
global_step=16490 episodic_return=[13.644708]
global_step=16501 episodic_return=[6.818907]
global_step=16513 episodic_return=[9.595403]
global_step=16524 episodic_return=[6.805711]
global_step=16556 episodic_return=[11.380069]
global_step=16571 episodic_return=[9.513685]
global_step=16597 episodic_return=[29.97246]
global_step=16609 episodic_return=[8.559511]
global_step=16630 episodic_return=[15.143441]
global_step=16734 episodic_return=[122.04427]
global_step=16757 

Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/videos/Hopper-v2__1__1700234374/rl-video-episode-1000.mp4
global_step=22373 episodic_return=[10.096142]
global_step=22384 episodic_return=[7.0258117]
global_step=22397 episodic_return=[3.7219362]
global_step=22420 episodic_return=[9.825436]
global_step=22454 episodic_return=[14.15803]
global_step=22465 episodic_return=[7.620246]
global_step=22489 episodic_return=[19.603174]
global_step=22515 episodic_return=[14.366137]
global_step=22525 episodic_return=[6.643969]
global_step=22537 episodic_return=[10.237014]
global_step=22553 episodic_return=[11.841006]
global_step=22572 episodic_return=[16.463182]
global_step=22618 episodic_return=[44.89943]
global_step=22640 episodic_return=[27.653717]
global_step=22654 episodic_return=[12.877375]
global_step=22669 episodic_return=[9.727212]
global_step=22709 episodic_return=[28.283169]
global_step=22725 episodic_return=[13.169201]
global_step=22762 episodic_return=[21.200922]
global_step=227

Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/videos/Hopper-v2__1__1700234374/rl-video-episode-2000.mp4
global_step=85657 episodic_return=[337.43048]
global_step=85811 episodic_return=[322.03302]
global_step=85990 episodic_return=[351.85638]
global_step=86159 episodic_return=[337.5193]
global_step=86322 episodic_return=[334.69858]
global_step=86487 episodic_return=[332.96228]
global_step=86643 episodic_return=[323.6351]
global_step=86800 episodic_return=[322.49625]
global_step=86955 episodic_return=[319.60608]
global_step=87115 episodic_return=[325.00436]
global_step=87278 episodic_return=[330.36307]
global_step=87411 episodic_return=[296.64694]
global_step=87571 episodic_return=[326.2152]
global_step=87698 episodic_return=[288.51697]
global_step=87846 episodic_return=[306.14145]
global_step=87987 episodic_return=[302.80508]
global_step=88119 episodic_return=[300.62012]
global_step=88272 episodic_return=[325.4854]
global_step=88406 episodic_return=[305.50104]
global_step=8

Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/videos/Hopper-v2__1__1700234374/rl-video-episode-3000.mp4
global_step=329282 episodic_return=[284.38712]
global_step=329435 episodic_return=[286.03583]
global_step=329592 episodic_return=[296.95917]
global_step=329741 episodic_return=[289.4596]
global_step=329897 episodic_return=[298.84033]
global_step=330072 episodic_return=[371.979]
global_step=330221 episodic_return=[294.2175]
global_step=330389 episodic_return=[307.76233]
global_step=330538 episodic_return=[281.09015]
global_step=330692 episodic_return=[280.32526]
global_step=330853 episodic_return=[311.59134]
global_step=331013 episodic_return=[320.65845]
global_step=331165 episodic_return=[294.75766]
global_step=331313 episodic_return=[285.3443]
global_step=331460 episodic_return=[289.61658]
global_step=331613 episodic_return=[302.57736]
global_step=331761 episodic_return=[305.0718]
global_step=331908 episodic_return=[298.54553]
global_step=332059 episodic_return=[300.086

charts/SPS,█▅▄▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
charts/episodic_length,▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▃▁▂▃▃▃▂▃▃▂▂▃▂▁▄▅█▃▄▅▆▅
charts/episodic_return,▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂▃▄▁▁▃▄▃▂▄▃▂▃▄▃▁▄▆▇▃▄██▇
global_step,▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇██
losses/actor_loss,██▇▆▅▄▃▃▃▂▂▂▁▁▁▂▂▂▃▃▄▄▄▃▃▃▄▄▄▄▃▃▃▃▄▄▄▄▄▄
losses/qf1_loss,▁▁▂▃▆▇▆▇█▄▆▄▃▂▃▄▂▁▂▂▁▁▁▂▂▁▁▁▁▁▁▁▁▄▁▁▁▁▁▁
losses/qf1_values,▁▁▂▃▄▅▆▆▆▇▇▇███▇▇▇▇▆▅▅▅▆▆▆▅▅▅▆▆▆▆▆▅▅▅▅▅▅
charts/SPS,111.0
charts/episodic_length,204.0
charts/episodic_return,499.40213
global_step,499964
